# Streaming Results

In [ ]:
!python -m pip install semantic-kernel==0.3.10.dev0

In [ ]:
import semantic_kernel as sk
from semantic_kernel.connectors.ai import ChatRequestSettings, CompleteRequestSettings
from semantic_kernel.connectors.ai.open_ai import AzureTextCompletion, AzureChatCompletion, OpenAITextCompletion, OpenAIChatCompletion
from semantic_kernel.connectors.ai.hugging_face import HuggingFaceTextCompletion

First, we will set up the text and chat services we will be submitting prompts to.

In [ ]:
kernel = sk.Kernel()

# Configure Azure LLM service
deployment, api_key, endpoint = sk.azure_openai_settings_from_dot_env()
azure_text_service = AzureTextCompletion("text-davinci-003", endpoint, api_key)
azure_chat_service = AzureChatCompletion("gpt-35-turbo", endpoint, api_key)

# Configure OpenAI service
api_key, org_id = sk.openai_settings_from_dot_env()
oai_text_service = OpenAITextCompletion("text-davinci-003", api_key, org_id)
oai_chat_service = OpenAIChatCompletion("gpt-3.5-turbo", api_key, org_id)

# Configure Hugging Face service
hf_text_service = HuggingFaceTextCompletion("distilgpt2", task="text-generation")

Next, we'll set up the completion request settings for text completion services.

In [ ]:
request_settings = CompleteRequestSettings(
    max_tokens=150,
    temperature=0.7,
    top_p=1,
    frequency_penalty=0.5,
    presence_penalty=0.5
)

## Streaming Open AI Text Completion

In [ ]:
prompt = "what is the purpose of a rubber duck?"
stream = oai_text_service.complete_stream_async(prompt, request_settings)
async for text in stream:
    print(text, end = "") # end = "" to avoid newlines

## Streaming Azure Open AI Text Completion

In [ ]:
prompt = "provide me a list of possible meanings for the acronym 'ORLD'"
stream = azure_text_service.complete_stream_async(prompt, request_settings)
async for text in stream:
    print(text, end = "") # end = "" to avoid newlines

## Streaming Hugging Face Text Completion

In [ ]:
prompt = "The purpose of a rubber duck is"
stream = hf_text_service.complete_stream_async(prompt, request_settings)
async for text in stream:
    print(text, end = "") # end = "" to avoid newlines

Here, we're setting up the settings for Chat completions.

In [ ]:
chat_request_settings = ChatRequestSettings(
    max_tokens=150,
    temperature=0.7,
    top_p=1,
    frequency_penalty=0.5,
    presence_penalty=0.5,
)

## Streaming OpenAI Chat Completion

In [ ]:
prompt = "It's a beautiful day outside, birds are singing, flowers are blooming. On days like these, kids like you..."
stream = oai_chat_service.complete_chat_stream_async([("user", prompt)], chat_request_settings)
async for text in stream:
    print(text, end = "") # end = "" to avoid newlines

## Streaming Azure OpenAI Chat Completion

In [ ]:
prompt = "Tomorow is going to be a great day, I can feel it. I'm going to wake up early, go for a run, and then..."
stream = azure_chat_service.complete_chat_stream_async([("user", prompt)], chat_request_settings)
async for text in stream:
    print(text, end = "") # end = "" to avoid newlines